In [2]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
import matplotlib.pyplot as plt

In [4]:
from preprocessing.data_preprocessing import (
    WindowGenerator,
    split_dataset,
    get_normalized_dataset,
    load_data)

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
from StationarityTest.adf_test import StationaryTester

### Stationarity check

In [ ]:
def compile_and_fit(model, window, patience=10, n_epochs = 128):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError(), 'accuracy'])

  history = model.fit(window.train, epochs=n_epochs,
                      validation_data=window.val,)
                      # callbacks=[early_stopping])
  return history
  # return model

## Engine Timing model simulation data forecasting
Here we are considering the simulation data obtained from the above said model. In total we have 6 time series.

In [ ]:
ENGINE_TIMING_DATA_FILE_PATH = './data/Engine_Timing_sim_data_12_01_22_12_2022.xlsx'

engine_Timing_data_frame = load_data(ENGINE_TIMING_DATA_FILE_PATH, plot=True)
engine_Timing_data_frame.head()

In [ ]:
engine_Timing_data_frame.drop(columns=['time'], inplace=True)

In [ ]:
engine_Timing_data_frame.head()

In [ ]:
stat_tester = StationaryTester()
for col_name in engine_Timing_data_frame.columns:
    print('='*15 + f'Stationary test for {col_name}' + '='*15 + '\n')
    stat_tester.test(engine_Timing_data_frame[col_name].to_numpy(), has_trends=True)

In [ ]:
# ## Splitting the data
train_test_val_split = (0.8, 0.1, 0.1)
train_df, test_df, val_df = split_dataset(engine_Timing_data_frame, train_test_val_split)

# ## Normalization of data
train_df, val_df, test_df = get_normalized_dataset(train_df, val_df, test_df)


In [ ]:
engine_Timing_data_set = WindowGenerator(train_df=train_df, val_df=val_df, test_df=test_df,
                        input_width=24, label_width=24, shift=24)

In [ ]:
engine_Timing_data_set

In [ ]:
engine_Timing_data_set.plot()

In [ ]:
def build_model(num_features):
    model = tf.keras.models.Sequential([
        # Shape [batch, time, features] => [batch, time, lstm_units]
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        # Shape => [batch, time, features]
        tf.keras.layers.Dense(units=num_features)
    ])

    return model

In [ ]:
forecast_model = build_model(num_features=len(engine_Timing_data_set.train_df.columns))
forecast_model_history = compile_and_fit(forecast_model, engine_Timing_data_set)